### 1. Download dos arquivos tif do Anadem

In [ ]:
import os
import time
import requests
import urllib3
from http.client import IncompleteRead
from tqdm import tqdm

# Desabilita avisos chatos de SSL se houver
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def download_file_resume(url, save_path, max_retries=50): # Aumentei muito os retries
    """
    Baixa um arquivo com suporte a resume e é extremamente persistente.
    """
    # Define modo de escrita (append se já existe, write se é novo)
    if os.path.exists(save_path):
        current_size = os.path.getsize(save_path)
        resume_header = {'Range': f'bytes={current_size}-'}
        temp_mode = 'ab'
    else:
        resume_header = {}
        current_size = 0
        temp_mode = 'wb'

    retries = 0
    while retries < max_retries:
        try:
            # 1. Tenta pegar o tamanho total do arquivo
            try:
                head_resp = requests.head(url, timeout=30)
                if head_resp.status_code == 404:
                    print(f"ERRO 404: Arquivo não existe -> {url}")
                    return False
                total_file_size = int(head_resp.headers.get('content-length', 0))
            except:
                # Se falhar no HEAD, tentamos baixar direto para ver o que dá
                total_file_size = 0

            # Se já baixamos tudo, retorna sucesso
            if current_size >= total_file_size and total_file_size > 0:
                print(f"Arquivo já completo: {os.path.basename(save_path)}")
                return True

            # 2. Configura headers e faz o request
            headers = resume_header.copy()
            response = requests.get(url, headers=headers, stream=True, timeout=60)
            
            # Se servidor reiniciou o download (200) em vez de continuar (206)
            if response.status_code == 200:
                temp_mode = 'wb'
                current_size = 0
            elif response.status_code not in [200, 206]:
                raise requests.exceptions.RequestException(f"Status ruim: {response.status_code}")

            # 3. Baixa o conteúdo
            desc = os.path.basename(save_path)
            total_bar = total_file_size if total_file_size > 0 else None
            
            with open(save_path, temp_mode) as file, tqdm(
                desc=desc,
                total=total_bar,
                initial=current_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        size = file.write(chunk)
                        bar.update(size)
            
            # Se chegou aqui, sucesso!
            return True

        # --- BLOCO DE TRATAMENTO DE ERROS CORRIGIDO ---
        except (requests.exceptions.RequestException, 
                urllib3.exceptions.ProtocolError,
                urllib3.exceptions.ReadTimeoutError,
                IncompleteRead,
                ConnectionResetError) as e:
            
            print(f"\nConexão caiu ({type(e).__name__}). Retomando em 10s... ({retries+1}/{max_retries})")
            retries += 1
            time.sleep(10) # Espera um pouco mais para a rede estabilizar
            
            # Atualiza o ponteiro para retomar o download
            if os.path.exists(save_path):
                current_size = os.path.getsize(save_path)
                resume_header = {'Range': f'bytes={current_size}-'}
                temp_mode = 'ab'

        # --- O GOLEIRO: Pega qualquer outro erro inesperado ---
        except Exception as e:
            print(f"\nErro genérico não tratado ({e}). Tentando novamente em 10s...")
            retries += 1
            time.sleep(10)
            # Tenta retomar mesmo assim
            if os.path.exists(save_path):
                current_size = os.path.getsize(save_path)
                resume_header = {'Range': f'bytes={current_size}-'}
                temp_mode = 'ab'

    print(f"Falha definitiva após {max_retries} tentativas.")
    return False

def processar_lista(tiles_list, base_url, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)
    
    print(f"Salvando em: {os.path.abspath(output_folder)}\n")

    for tile in tiles_list:
        file_name = f"anadem_v1_{tile}.tif"
        url = f"{base_url}{file_name}"
        save_path = os.path.join(output_folder, file_name)
        
        success = download_file_resume(url, save_path)
        if not success:
            print(f"--> ERRO CRÍTICO: Não foi possível baixar {file_name}.\n")

# --- CONFIGURAÇÃO ---

lista_tiles = [
    "20P", "21P", "21N", "22M", "22N", "23M", "24M", "25M", 
    "25L", "24L", "24K", "23K", "23J", "22J", "22H"
]

url_base = "https://metadados.snirh.gov.br/files/anadem_v1_tiles/"
# Ajuste o caminho se necessário
pasta_destino = os.path.join("..", "data", "raw", "ana", "anadem")

if __name__ == "__main__":
    processar_lista(lista_tiles, url_base, pasta_destino)
    print("\nProcesso finalizado!")

Salvando em: c:\Users\CarolinaFaccin\OneDrive - World Resources Institute\Carolina WRI\Repositorios\climate-justice-index\data\raw\ana\anadem\original

Arquivo já completo: anadem_v1_20P.tif
Arquivo já completo: anadem_v1_21P.tif
Arquivo já completo: anadem_v1_21N.tif
Arquivo já completo: anadem_v1_22M.tif
Arquivo já completo: anadem_v1_22N.tif


anadem_v1_23M.tif: 100%|██████████| 1.51G/1.51G [01:54<00:00, 1.77MiB/s]
anadem_v1_24M.tif:  55%|█████▍    | 603M/1.07G [01:03<04:33, 1.90MiB/s]



Conexão caiu (ChunkedEncodingError). Retomando em 10s... (1/50)


anadem_v1_24M.tif: 100%|██████████| 1.07G/1.07G [04:13<00:00, 2.04MiB/s]


Arquivo já completo: anadem_v1_25M.tif
Arquivo já completo: anadem_v1_25L.tif


...


Conexão caiu (ChunkedEncodingError). Retomando em 10s... (1/50)


anadem_v1_22J.tif:  75%|███████▌  | 1.11G/1.47G [04:48<03:33, 1.82MiB/s] 



Conexão caiu (ChunkedEncodingError). Retomando em 10s... (2/50)


anadem_v1_22J.tif: 100%|██████████| 1.47G/1.47G [03:10<00:00, 2.04MiB/s]  


Arquivo já completo: anadem_v1_22H.tif

Processo finalizado!


### 2. Identificar municípios defronte com o mar

In [ ]:
import os
import pandas as pd
import geopandas as gpd

# --- CONFIGURAÇÃO DOS CAMINHOS ---
# Usando os caminhos relativos que você forneceu
input_gpkg = os.path.join("..", "data", "raw", "ibge", "BR_setores_CD2022_v2.gpkg")
input_csv = os.path.join("..", "data", "raw", "ibge", "malha_municipal_2024", "municipios_defrontantes_com_o_mar.csv")
output_gpkg = os.path.join("..", "data", "clean", "sc", "BR_setores_CD2022_mar.gpkg")

def processar_cruzamento():
    print("1. Lendo arquivo CSV de municípios costeiros...")
    # Lemos forçando o CD_MUN como string para garantir o match com o GPKG
    df_mar = pd.read_csv(
        input_csv, 
        sep=';', 
        dtype={'CD_MUN': str} 
    )
    
    # Garantir que a coluna alvo está limpa (caso venha como string "True")
    col_alvo_csv = 'municipios_defrontantes_com_o_mar'
    
    # Se o CSV tiver apenas os True, o resto será NaN no join.
    # Vamos manter apenas as colunas necessárias para o join ficar leve
    df_mar = df_mar[['CD_MUN', col_alvo_csv]]

    print("2. Lendo arquivo GPKG dos Setores Censitários (Isso pode demorar um pouco)...")
    gdf_setores = gpd.read_file(input_gpkg)
    
    # Garantir que CD_MUN do GeoDataFrame também seja string
    if 'CD_MUN' in gdf_setores.columns:
        gdf_setores['CD_MUN'] = gdf_setores['CD_MUN'].astype(str)
    else:
        raise ValueError("A coluna CD_MUN não foi encontrada no GPKG.")

    print(f"   - Total de setores carregados: {len(gdf_setores)}")

    print("3. Realizando o cruzamento (Left Join)...")
    # Join pela coluna CD_MUN
    gdf_merged = gdf_setores.merge(df_mar, on='CD_MUN', how='left')

    # Tratamento da nova coluna:
    # Onde deu match é True, onde não deu (NaN) vira False
    gdf_merged['mun_defronte_mar'] = gdf_merged[col_alvo_csv].fillna(False).astype(bool)

    # Remove a coluna original do CSV para não ficar duplicada ou com nome feio
    gdf_merged = gdf_merged.drop(columns=[col_alvo_csv])

    print("4. Salvando novo arquivo GPKG...")
    # Salvando no novo caminho
    gdf_merged.to_file(output_gpkg, driver='GPKG')
    
    print(f"--- Sucesso! ---")
    print(f"Arquivo salvo em: {output_gpkg}")
    print(f"Verificação rápida:")
    print(gdf_merged[['CD_MUN', 'mun_defronte_mar']].head())
    print(f"\nContagem de setores em municípios costeiros: {gdf_merged['mun_defronte_mar'].sum()}")

if __name__ == "__main__":
    processar_cruzamento()

1. Lendo arquivo CSV de municípios costeiros...
2. Lendo arquivo GPKG dos Setores Censitários (Isso pode demorar um pouco)...
   - Total de setores carregados: 472780
3. Realizando o cruzamento (Left Join)...


C:\Users\CarolinaFaccin\AppData\Local\Temp\ipykernel_22788\2688384629.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf_merged['mun_defronte_mar'] = gdf_merged[col_alvo_csv].fillna(False).astype(bool)


4. Salvando novo arquivo GPKG...
--- Sucesso! ---
Arquivo salvo em: ..\data\clean\sc\BR_setores_CD2022_mar.gpkg
Verificação rápida:
    CD_MUN  mun_defronte_mar
0  1100015             False
1  1100015             False
2  1100015             False
3  1100015             False
4  1100015             False

Contagem de setores em municípios costeiros: 83940


### 3. Criar o Mosaico Virtual (VRT) dos tiles do Anadem

In [ ]:
import os
import glob
from osgeo import gdal

# Caminhos
input_folder = os.path.join("..", "data", "raw", "ana", "anadem", "original")
output_vrt = os.path.join("..", "data", "raw", "ana", "anadem", "virtual", "anadem_litoral_virtual.vrt")

# Lista todos os TIFs baixados
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))

print(f"Encontrados {len(tif_files)} arquivos TIF.")

# Cria o VRT (Virtual Raster)
# Isso leva segundos, pois não processa pixels, apenas metadados.
gdal.BuildVRT(output_vrt, tif_files)

print(f"Mosaico virtual criado em: {output_vrt}")

Encontrados 15 arquivos TIF.
Mosaico virtual criado em: ..\data\raw\ana\anadem\virtual\anadem_litoral_virtual.vrt


### 4. Setores: Loop para calcular a elevação do nivel do mar e inputar os dados nos setores censitários

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.mask import mask
from rasterio.features import shapes
from shapely.geometry import shape, box
from tqdm import tqdm
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# --- CONFIGURAÇÕES ---
vrt_path = os.path.join("..", "data", "raw", "ana", "anadem", "virtual", "anadem_litoral_virtual.vrt")
input_setores = os.path.join("..", "data", "clean", "sc", "BR_setores_CD2022_mar.gpkg")
# REMOVIDO: output_parquet (não vamos mais usar)
output_gpkg = os.path.join("..", "data", "clean", "sc", "BR_setores_CD2022_e3.gpkg")

def calcular_exposicao_costeira():
    print("Carregando setores censitários...")
    gdf_all = gpd.read_file(input_setores)
    
    if 'CD_MUN' in gdf_all.columns:
        gdf_all['CD_MUN'] = gdf_all['CD_MUN'].astype(str)

    # Filtrar apenas setores de municípios costeiros
    gdf_costa = gdf_all[gdf_all['mun_defronte_mar'] == True].copy()
    gdf_costa = gdf_costa.reset_index(drop=True)
    
    municipios_unicos = gdf_costa['CD_MUN'].unique()
    print(f"Total de municípios costeiros para processar: {len(municipios_unicos)}")

    resultados_lista = []

    with rasterio.open(vrt_path) as src_raster:
        for cd_mun in tqdm(municipios_unicos, desc="Processando Municípios"):
            try:
                gdf_mun = gdf_costa[gdf_costa['CD_MUN'] == cd_mun].copy()
                if gdf_mun.empty: continue

                # Recorte e Identificação de Risco
                bbox_mun = box(*gdf_mun.total_bounds)
                try:
                    out_image, out_transform = mask(src_raster, [bbox_mun], crop=True)
                except ValueError:
                    continue 

                dados_altimetria = out_image[0]
                mask_risco = (dados_altimetria <= 1.0) & (dados_altimetria > -50)

                if not mask_risco.any(): continue

                geoms_risco = []
                for geom, val in shapes(mask_risco.astype('int16'), mask=mask_risco, transform=out_transform):
                    geoms_risco.append(shape(geom))
                
                if not geoms_risco: continue

                gdf_risco = gpd.GeoDataFrame({'geometry': geoms_risco}, crs=gdf_mun.crs)
                gdf_mask = gdf_risco.dissolve()
                gdf_mask['geometry'] = gdf_mask.geometry.buffer(0)

                # Intersecção
                interseccao = gpd.overlay(gdf_mun.reset_index(drop=True), 
                                          gdf_mask.reset_index(drop=True), 
                                          how='intersection')

                if interseccao.empty: continue

                # Cálculos de Área
                gdf_mun_proj = gdf_mun.to_crs(epsg=5880)
                areas_totais = gdf_mun_proj.set_index('CD_SETOR').geometry.area

                inter_proj = interseccao.to_crs(epsg=5880)
                inter_proj['area_alagada_m2'] = inter_proj.geometry.area
                areas_alagadas = inter_proj.groupby('CD_SETOR')['area_alagada_m2'].sum()

                for cd_setor, area_alagada in areas_alagadas.items():
                    area_total = areas_totais.get(cd_setor, np.nan)
                    pct = 0.0
                    if pd.notna(area_total) and area_total > 0:
                        pct = (area_alagada / area_total) * 100
                        pct = min(pct, 100.0)

                    resultados_lista.append({
                        'CD_SETOR': cd_setor,
                        'area_alagada_m2': area_alagada,
                        'pct_area_impactada_1m': pct
                    })

            except Exception as e:
                continue

    # --- SALVAMENTO (MODIFICADO) ---
    print("Consolidando resultados...")
    if not resultados_lista:
        print("Nenhum resultado gerado.")
        return

    df_resultados = pd.DataFrame(resultados_lista)
    
    # Agrega duplicatas
    df_resultados = df_resultados.groupby('CD_SETOR', as_index=False).agg({
        'area_alagada_m2': 'max',
        'pct_area_impactada_1m': 'max'
    })
    
    # 1. Backup em CSV (Muito mais seguro que Parquet)
    csv_backup = os.path.join("..", "data", "clean", "sc", "temp_resultados_e3.csv")
    df_resultados.to_csv(csv_backup, sep=';', index=False)
    print(f"Backup CSV salvo em: {csv_backup}")

    print("Criando GPKG final (E3)...")
    
    # Garante tipagem para o merge
    gdf_all['CD_SETOR'] = gdf_all['CD_SETOR'].astype(str)
    df_resultados['CD_SETOR'] = df_resultados['CD_SETOR'].astype(str)

    gdf_final = gdf_all.merge(df_resultados, on='CD_SETOR', how='left')
    gdf_final[['pct_area_impactada_1m', 'area_alagada_m2']] = gdf_final[['pct_area_impactada_1m', 'area_alagada_m2']].fillna(0)
    
    gdf_final.to_file(output_gpkg, driver='GPKG')
    print(f"Sucesso! Arquivo final salvo em: {output_gpkg}")

if __name__ == "__main__":
    calcular_exposicao_costeira()

Carregando setores censitários...
Total de municípios costeiros para processar: 279


Processando Municípios: 100%|██████████| 279/279 [15:03<00:00,  3.24s/it]   


Consolidando resultados...
Backup CSV salvo em: ..\data\clean\sc\temp_resultados_e3.csv
Criando GPKG final (E3)...
Sucesso! Arquivo final salvo em: ..\data\clean\sc\BR_setores_CD2022_e3.gpkg


### 5. Setores: Update do '_e3.gpkg'
Este código pega o seu arquivo já pronto e zera os valores dos setores que estão longe da costa.

In [ ]:


import os
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# --- CONFIGURAÇÕES ---
# O arquivo alvo (que será lido e sobrescrito)
arquivo_alvo = os.path.join("..", "data", "clean", "sc", "BR_setores_CD2022_e3.gpkg")
input_oceano = os.path.join("..", "data", "raw", "ibge", "hidrografia_2017", "oceano_atlantico.gpkg")

def atualizar_arquivo_existente():
    print(f"1. Lendo arquivo alvo: {arquivo_alvo} ...")
    gdf_setores = gpd.read_file(arquivo_alvo)
    
    # Garante que CD_SETOR é string para evitar problemas de compatibilidade
    if 'CD_SETOR' in gdf_setores.columns:
        gdf_setores['CD_SETOR'] = gdf_setores['CD_SETOR'].astype(str)

    # --- ETAPA DE FILTRAGEM (Spatial Filter) ---
    print("2. Calculando filtro de conectividade com o oceano...")
    
    # Seleciona apenas setores com algum risco > 0 para testar a proximidade (otimização)
    mask_com_risco = gdf_setores['pct_area_impactada_1m'] > 0
    gdf_risco_teste = gdf_setores[mask_com_risco].copy()
    
    if gdf_risco_teste.empty:
        print("   - Nenhum setor com risco encontrado. Nada a filtrar.")
        ids_validos = []
    else:
        # Carrega e prepara o oceano
        gdf_oceano = gpd.read_file(input_oceano)
        if gdf_oceano.crs != gdf_setores.crs:
            gdf_oceano = gdf_oceano.to_crs(gdf_setores.crs)

        # Define tamanho do buffer
        if gdf_setores.crs.is_geographic:
            buffer_size = 0.005 # ~500m
        else:
            buffer_size = 500 # 500m

        # Cria buffer único do oceano
        geom_oceano = gdf_oceano.dissolve().geometry.buffer(buffer_size).iloc[0]
        gdf_oceano_buffer = gpd.GeoDataFrame({'geometry': [geom_oceano]}, crs=gdf_setores.crs)

        # Verifica intersecção
        gdf_conectados = gpd.sjoin(gdf_risco_teste, gdf_oceano_buffer, how='inner', predicate='intersects')
        ids_validos = gdf_conectados['CD_SETOR'].unique()

    print(f"   - Setores validados como costeiros: {len(ids_validos)}")

    # --- ETAPA DE CRIAÇÃO DAS COLUNAS ---
    print("3. Criando novas colunas...")

    # A. Cria a coluna LIMPA
    # Primeiro copiamos tudo da original
    gdf_setores['pct_area_impactada_1m_limpo'] = gdf_setores['pct_area_impactada_1m']
    
    # Identificamos quem deve ser zerado (Tem risco > 0 MAS NÃO está na lista de válidos)
    mask_zerar = (gdf_setores['pct_area_impactada_1m'] > 0) & (~gdf_setores['CD_SETOR'].isin(ids_validos))
    
    # Aplica o zero
    gdf_setores.loc[mask_zerar, 'pct_area_impactada_1m_limpo'] = 0.0
    print(f"   - Valores zerados por estarem no interior: {mask_zerar.sum()} setores.")

    # B. Cria a coluna NORMALIZADA (0 a 1)
    # Usa a coluna JÁ LIMPA como base
    gdf_setores['e3_mar_norm'] = gdf_setores['pct_area_impactada_1m_limpo'] / 100.0
    
    # Garante que não passe de 1.0 (caso haja erro de arredondamento)
    gdf_setores.loc[gdf_setores['e3_mar_norm'] > 1.0, 'e3_mar_norm'] = 1.0

    # --- SALVAMENTO ---
    print(f"4. Sobrescrevendo arquivo: {arquivo_alvo} ...")
    # O GeoPandas carrega em memória, então é seguro salvar sobre o mesmo arquivo
    gdf_setores.to_file(arquivo_alvo, driver='GPKG')
    
    print("--- Sucesso! ---")
    print("Novas colunas adicionadas:")
    print("1. 'pct_area_impactada_1m_limpo' (Percentual filtrado)")
    print("2. 'e3_mar_norm' (Normalizado 0-1)")

if __name__ == "__main__":
    atualizar_arquivo_existente()

1. Lendo arquivo alvo: ..\data\clean\sc\BR_setores_CD2022_e3.gpkg ...
2. Calculando filtro de conectividade com o oceano...
   - Setores validados como costeiros: 2043
3. Criando novas colunas...
   - Valores zerados por estarem no interior: 4085 setores.
4. Sobrescrevendo arquivo: ..\data\clean\sc\BR_setores_CD2022_e3.gpkg ...
--- Sucesso! ---
Novas colunas adicionadas:
1. 'pct_area_impactada_1m_limpo' (Percentual filtrado)
2. 'e3_mar_norm' (Normalizado 0-1)


### 6. Setores: Salva novo arquivo BR_setores_CD2022_e1235_vul_int.gpkg com a coluna 'e3_mar_norm'

In [ ]:
import os
import geopandas as gpd
import pandas as pd

# --- CONFIGURAÇÕES DOS ARQUIVOS ---
# Arquivo BASE (que receberá a coluna)
input_base_path = os.path.join("..", "data", "clean", "sc", "BR_setores_CD2022_e125_vul_int.gpkg")

# Arquivo ORIGEM (de onde vem a coluna e3_mar_norm)
input_source_path = os.path.join("..", "data", "clean", "sc", "BR_setores_CD2022_e3.gpkg")

# Arquivo FINAL (Output)
output_path = os.path.join("..", "data", "clean", "sc", "BR_setores_CD2022_e1235_vul_int.gpkg")

def consolidar_indicador_e3():
    print("1. Carregando arquivo BASE (e1, e2, e5, vul_int)...")
    gdf_base = gpd.read_file(input_base_path)
    
    # Garante tipagem da chave
    if 'CD_SETOR' in gdf_base.columns:
        gdf_base['CD_SETOR'] = gdf_base['CD_SETOR'].astype(str)
    
    print(f"   - Total de setores na base: {len(gdf_base)}")

    print("2. Carregando dados do INDICADOR E3 (Nível do Mar)...")
    # Truque de performance: ignore_geometry=True faz carregar como DataFrame normal (muito mais rápido)
    # Se sua versão do geopandas for antiga e der erro, remova o ignore_geometry=True
    try:
        df_source = gpd.read_file(input_source_path, ignore_geometry=True)
    except TypeError:
        # Fallback para versões antigas
        df_source = gpd.read_file(input_source_path)
        df_source = pd.DataFrame(df_source) # Remove parte geo

    # Seleciona apenas o que importa para o merge
    colunas_interesse = ['CD_SETOR', 'e3_mar_norm']
    
    # Verifica se a coluna existe mesmo
    if 'e3_mar_norm' not in df_source.columns:
        raise ValueError("A coluna 'e3_mar_norm' não foi encontrada no arquivo de origem!")
        
    df_source = df_source[colunas_interesse].copy()
    df_source['CD_SETOR'] = df_source['CD_SETOR'].astype(str)
    
    print(f"   - Dados de E3 carregados para {len(df_source)} setores.")

    print("3. Realizando o cruzamento (Merge)...")
    # Left Join: Mantém todos da base. Quem não tiver correspondência no E3 vira NaN.
    gdf_merged = gdf_base.merge(df_source, on='CD_SETOR', how='left')
    
    # Preenchimento de Nulos:
    # Se o setor não estava no arquivo E3 (interior do país), o risco marinho é 0.
    nulos_antes = gdf_merged['e3_mar_norm'].isna().sum()
    gdf_merged['e3_mar_norm'] = gdf_merged['e3_mar_norm'].fillna(0.0)
    
    print(f"   - Merge concluído. {nulos_antes} setores do interior receberam valor 0.0.")

    print("4. Salvando novo arquivo consolidado...")
    gdf_merged.to_file(output_path, driver='GPKG')
    
    print(f"--- SUCESSO! ---")
    print(f"Arquivo salvo em: {output_path}")
    print("Colunas presentes:", list(gdf_merged.columns))

if __name__ == "__main__":
    consolidar_indicador_e3()

1. Carregando arquivo BASE (e1, e2, e5, vul_int)...
   - Total de setores na base: 468099
2. Carregando dados do INDICADOR E3 (Nível do Mar)...
   - Dados de E3 carregados para 472780 setores.
3. Realizando o cruzamento (Merge)...
   - Merge concluído. 0 setores do interior receberam valor 0.0.
4. Salvando novo arquivo consolidado...
--- SUCESSO! ---
Arquivo salvo em: ..\data\clean\sc\BR_setores_CD2022_e1235_vul_int.gpkg
Colunas presentes: ['CD_SETOR', 'id', 'SITUACAO', 'CD_SIT', 'CD_TIPO', 'AREA_KM2', 'CD_REGIAO', 'NM_REGIAO', 'CD_UF', 'NM_UF', 'CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST', 'CD_BAIRRO', 'NM_BAIRRO', 'CD_NU', 'NM_NU', 'CD_FCU', 'NM_FCU', 'CD_AGLOM', 'NM_AGLOM', 'CD_RGINT', 'NM_RGINT', 'CD_RGI', 'NM_RGI', 'CD_CONCURB', 'NM_CONCURB', 'total_enderecos', 'total_enderecos_favelas', 'v2_p_enderecos_fcu', 'e5_media_historica_secas', 'e5_sec_norm', 'v2_mor_norm', 'V00853', 'V00855', 'V00857', 'V00001', 'V00201', 'V00238', 'V01318', 'V01319', 'V01320', 'V

---

### 7. Malha H3: Inputar dados do ANADEM na malha H3

In [14]:
import os
import glob
from osgeo import gdal
import rasterio

# --- CONFIGURAÇÕES ---
# Caminho da pasta onde estão os TIFs baixados (Raw)
pasta_tifs = os.path.join("..", "data", "raw", "ana", "anadem", "original")
# Caminho onde salvaremos o VRT robusto
output_vrt = os.path.join("..", "data", "raw", "ana", "anadem", "virtual", "anadem_litoral_absoluto.vrt")

def reparar_vrt():
    # 1. Obter caminhos ABSOLUTOS dos TIFs
    # O abspath resolve o problema de "onde estou rodando o script"
    search_path = os.path.join(os.path.abspath(pasta_tifs), "*.tif")
    tif_files = glob.glob(search_path)
    
    print(f"Buscando TIFs em: {search_path}")
    print(f"Arquivos encontrados: {len(tif_files)}")
    
    if len(tif_files) == 0:
        print("ERRO CRÍTICO: Nenhum arquivo .tif encontrado. Verifique o caminho da pasta!")
        return

    # 2. Criar diretório do VRT se não existir
    pasta_vrt = os.path.dirname(output_vrt)
    if not os.path.exists(pasta_vrt):
        os.makedirs(pasta_vrt)

    # 3. Construir o VRT
    print(f"Criando VRT com caminhos absolutos em: {os.path.abspath(output_vrt)}")
    
    # gdal.BuildVRT aceita lista de strings
    options = gdal.BuildVRTOptions(resampleAlg='nearest', addAlpha=False)
    vrt_ds = gdal.BuildVRT(output_vrt, tif_files, options=options)
    vrt_ds = None # Salva e fecha o arquivo
    
    print("VRT Criado com sucesso.")

    # 4. TESTE DE AGULHA (Probe)
    # Vamos testar um ponto no Rio de Janeiro (Aeroporto Santos Dumont)
    # Lat: -22.91, Lon: -43.16. Altitude deve ser baixa (< 5m).
    rio_lon, rio_lat = -43.16, -22.91
    
    print("\n--- TESTE DE LEITURA (PROBE) ---")
    try:
        with rasterio.open(output_vrt) as src:
            print(f"CRS do VRT: {src.crs}")
            print(f"Limites do Raster: {src.bounds}")
            
            # Tenta ler o ponto. O Rasterio espera (Lon, Lat) se o CRS for Geo
            # Se for UTM, esse sample vai dar erro ou nodata, mas vamos testar.
            try:
                val_gen = src.sample([(rio_lon, rio_lat)])
                valor = list(val_gen)[0][0]
                print(f"Leitura no Rio de Janeiro ({rio_lon}, {rio_lat}): {valor} metros")
                
                if valor == src.nodata or valor < -100:
                    print("ALERTA: O teste retornou NoData. Verifique se o tile do RJ (23K ou 23J) foi baixado.")
                else:
                    print("SUCESSO: O Raster está lendo dados reais!")
            except Exception as e:
                print(f"Erro ao tentar ler amostra: {e}")
                
    except Exception as e:
        print(f"Erro ao abrir o VRT: {e}")

if __name__ == "__main__":
    reparar_vrt()

Buscando TIFs em: c:\Users\CarolinaFaccin\OneDrive - World Resources Institute\Carolina WRI\Repositorios\climate-justice-index\data\raw\ana\anadem\original\*.tif
Arquivos encontrados: 15
Criando VRT com caminhos absolutos em: c:\Users\CarolinaFaccin\OneDrive - World Resources Institute\Carolina WRI\Repositorios\climate-justice-index\data\raw\ana\anadem\virtual\anadem_litoral_absoluto.vrt
VRT Criado com sucesso.

--- TESTE DE LEITURA (PROBE) ---
CRS do VRT: EPSG:4326
Limites do Raster: BoundingBox(left=-66.00137988094313, bottom=-35.22428976325263, right=-33.90322730642645, top=14.079475111062084)
Leitura no Rio de Janeiro (-43.16, -22.91): 0.0 metros
SUCESSO: O Raster está lendo dados reais!


---

### 8. Malha H3: Gerar parquet da malha H3 com input dos dados de elevação do nível do mar
Este código calcula o indicador de Suscetibilidade à Elevação do Nível do Mar (e3) para a malha hexagonal H3 (Resolução 9) contendo domicílios. O script classifica como Área de Risco (e3_mar = 1) os hexágonos que atendem simultaneamente a dois critérios:

1. Altimetria: Possuem elevação do terreno entre 0 e 1 metro (dados extraídos do MDE ANADEM via amostragem pontual dos centroides).

2. Conectividade Costeira: Estão situados a uma distância de até 1.000 metros da linha de costa, incluindo Oceano, Lagunas e Canais (validado via intersecção espacial com o arquivo de buffer pré-processado).

Hexágonos que não atendem a ambos os critérios (ex: estão baixos mas no interior do continente, ou estão no litoral mas em morros) recebem valor 0.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.warp import transform
import h3
from tqdm import tqdm
import numpy as np
from shapely.geometry import Point

# --- CONFIGURAÇÕES ---
vrt_path = os.path.join("..", "data", "raw", "ana", "anadem", "virtual", "anadem_litoral_absoluto.vrt")
input_h3_parquet = os.path.join("..", "data", "clean", "h3", "br", "br_h3_res9_com_enderecos.parquet")
# Arquivo do oceano já com buffer feito no QGIS
input_oceano_buffer = os.path.join("..", "data", "raw", "ibge", "hidrografia_2017", "br_oceano_lagunas_canais_buffer_1000m.gpkg")

output_h3_parquet = os.path.join("..", "data", "clean", "h3", "br", "br_h3_res9_e3_mar.parquet")

def processar_h3_final_limpo():
    print(f"1. Carregando H3: {input_h3_parquet}")
    df_h3 = pd.read_parquet(input_h3_parquet)
    df_h3['e3_mar'] = 0 
    
    # Identificação robusta da Coluna ID
    col_h3 = 'h3_id'
    if col_h3 not in df_h3.columns:
        if 'h3_address' in df_h3.columns: col_h3 = 'h3_address'
        elif df_h3.index.name == 'h3_id': df_h3 = df_h3.reset_index()
        else: return print("Erro: Coluna ID H3 não encontrada.")

    # --- 2. EXTRAÇÃO DE COORDENADAS ---
    print("2. Convertendo IDs H3 para Lat/Lon...")
    
    def resolver_h3_geo(valor):
        s = str(valor).strip()
        # Se for numérico, converte para Hex String antes de pedir a geo
        if s.isdigit():
            try: return h3.cell_to_latlng(h3.int_to_str(int(s)))
            except: pass
        try: return h3.cell_to_latlng(s)
        except: return None, None

    lats, lons, indices_validos = [], [], []
    vals = df_h3[col_h3].values
    
    for idx, val in tqdm(enumerate(vals), total=len(vals), desc="Extraindo Geo"):
        lat, lon = resolver_h3_geo(val)
        if lat is not None:
            lats.append(lat)
            lons.append(lon)
            indices_validos.append(idx)
    
    if not indices_validos: return print("Erro: Nenhuma coordenada extraída.")

    # --- 3. AMOSTRAGEM ANADEM ---
    print("3. Amostrando Altitude (ANADEM)...")
    altitudes = np.full(len(df_h3), -9999.0)

    if not os.path.exists(vrt_path): return print(f"Erro: VRT não encontrado em {vrt_path}")

    with rasterio.open(vrt_path) as src:
        xs, ys = lons, lats
        # Reprojeção automática para o sistema do raster se necessário
        if src.crs != 'EPSG:4326' and src.crs != 'EPSG:4674':
            xs, ys = transform('EPSG:4326', src.crs, lons, lats)
        
        coords = list(zip(xs, ys))
        batch_size = 500000
        resultados = []
        
        for i in tqdm(range(0, len(coords), batch_size), desc="Lendo Pixels"):
            gen = src.sample(coords[i : i+batch_size], indexes=1)
            resultados.extend([x[0] for x in gen])
            
        for i, val in enumerate(resultados):
            altitudes[indices_validos[i]] = val

    df_h3['elevacao_m'] = altitudes

    # --- 4. FILTRAGEM ---
    print("4. Filtrando Candidatos (0 <= Alt <= 1)...")
    mask_cand = (df_h3['elevacao_m'] >= 0.0) & (df_h3['elevacao_m'] <= 1.0)
    num_cand = mask_cand.sum()
    print(f"   - Hexágonos candidatos: {num_cand}")

    if num_cand > 0:
        print(f"5. Validando Conectividade (Arquivo Buffer)...")
        gdf_buffer = gpd.read_file(input_oceano_buffer)
        
        # Prepara geometria apenas dos candidatos para otimizar
        df_c = df_h3[mask_cand].copy()
        
        geoms_c = []
        for idx in df_c.index:
            val = df_c.loc[idx, col_h3]
            lat, lon = resolver_h3_geo(val)
            geoms_c.append(Point(lon, lat)) 
            
        # Cria GDF em WGS84
        gdf_c = gpd.GeoDataFrame(df_c, geometry=geoms_c, crs="EPSG:4326")
        
        # Alinha projeção com o arquivo de buffer (seja ele qual for)
        if gdf_c.crs != gdf_buffer.crs:
            print(f"   - Ajustando projeção dos pontos para {gdf_buffer.crs}...")
            gdf_c = gdf_c.to_crs(gdf_buffer.crs)

        # Cruzamento Espacial
        print("   - Cruzando (Sjoin)...")
        gdf_join = gpd.sjoin(gdf_c, gdf_buffer, predicate='intersects')
        
        ids_confirmados = gdf_join[col_h3].unique()
        print(f"   - Hexágonos CONFIRMADOS: {len(ids_confirmados)}")
        
        # Atualiza coluna final
        df_h3.loc[df_h3[col_h3].isin(ids_confirmados), 'e3_mar'] = 1

    # --- 5. SALVAMENTO ---
    print("6. Salvando Parquet Final...")
    # Formatação de ID para String Hex (Padronização)
    df_h3[col_h3] = df_h3[col_h3].apply(lambda x: h3.int_to_str(int(x)) if str(x).isdigit() else str(x))
    
    df_h3.to_parquet(output_h3_parquet)
    print("Concluído.")
    print(df_h3['e3_mar'].value_counts())

if __name__ == "__main__":
    processar_h3_final_limpo()

1. Carregando H3: ..\data\clean\h3\br\br_h3_res9_com_enderecos.parquet
2. Convertendo IDs H3 para Lat/Lon...


Extraindo Geo: 100%|██████████| 4896048/4896048 [00:03<00:00, 1437507.52it/s]


3. Amostrando Altitude (ANADEM)...


Lendo Pixels: 100%|██████████| 10/10 [03:55<00:00, 23.56s/it]


4. Filtrando Candidatos (0 <= Alt <= 1)...
   - Hexágonos candidatos: 21860
5. Validando Conectividade (Arquivo Buffer)...
   - Ajustando projeção dos pontos para EPSG:5880...
   - Cruzando (Sjoin)...
   - Hexágonos CONFIRMADOS: 4442
6. Salvando Parquet Final...
Concluído.
e3_mar
0    4891606
1       4442
Name: count, dtype: int64


### 9. Malha H3: Ler parquet para testar 

In [ ]:
import pandas as pd
import os

# Caminho do arquivo final que você gerou
parquet_path = os.path.join("..", "data", "clean", "h3", "br", "br_h3_res9_e3_mar.parquet")

def verificar_resultado():
    print(f"Lendo arquivo: {parquet_path}")
    
    if not os.path.exists(parquet_path):
        print("Erro: Arquivo não encontrado.")
        return

    df = pd.read_parquet(parquet_path)
    
    print("\n--- CONTAGEM DE VALORES NA COLUNA 'e3_mar' ---")
    contagem = df['e3_mar'].value_counts()
    print(contagem)
    
    print("\n--- ESTATÍSTICAS DA COLUNA 'elevacao_m' PARA OS CASOS DE RISCO ---")
    # Filtra só quem é risco 1 para ver se a altitude faz sentido
    risco = df[df['e3_mar'] == 1]
    print(risco['elevacao_m'].describe())

if __name__ == "__main__":
    verificar_resultado()

Lendo arquivo: ..\data\clean\h3\br\br_h3_res9_e3_mar.parquet

--- CONTAGEM DE VALORES NA COLUNA 'e3_mar' ---
e3_mar
0    4891606
1       4442
Name: count, dtype: int64

--- ESTATÍSTICAS DA COLUNA 'elevacao_m' PARA OS CASOS DE RISCO ---
count    4442.000000
mean        0.221344
std         0.325272
min         0.000000
25%         0.000000
50%         0.000000
75%         0.500000
max         1.000000
Name: elevacao_m, dtype: float64


### 10. Malha H3: Merge das camadas '_e3_mar.parquet' com '_e125_vul_int.parquet'

In [ ]:
import os
import pandas as pd

# --- CONFIGURAÇÕES ---
input_base = os.path.join("..", "data", "clean", "h3", "br", "br_h3_res9_e125_vul_int.parquet")
input_e3 = os.path.join("..", "data", "clean", "h3", "br", "br_h3_res9_e3_mar.parquet")
output_final = os.path.join("..", "data", "clean", "h3", "br", "br_h3_res9_e1235_vul_int.parquet")

def consolidar_h3_final():
    print(f"1. Lendo base (e1, e2, e5, vul_int): {input_base}")
    df_base = pd.read_parquet(input_base)
    
    # Normalização de ID (Base)
    col_id_base = 'h3_id'
    if col_id_base not in df_base.columns:
        # Tenta achar nomes comuns
        found = False
        for c in ['h3_address', 'hex_id', 'h3_index']:
            if c in df_base.columns:
                df_base.rename(columns={c: 'h3_id'}, inplace=True)
                found = True
                break
        if not found and df_base.index.name in ['h3_id', 'h3_address', 'h3_index']:
            df_base = df_base.reset_index()
            # Se o index não tinha nome, renomeia a coluna criada 'index'
            if 'h3_id' not in df_base.columns: df_base.rename(columns={'index': 'h3_id'}, inplace=True)

    print(f"   - Total de hexágonos na base: {len(df_base)}")

    print(f"2. Lendo indicador E3 (Nível do Mar): {input_e3}")
    df_e3 = pd.read_parquet(input_e3)
    
    # Normalização de ID (E3)
    col_id_e3 = 'h3_id'
    if col_id_e3 not in df_e3.columns:
        for c in ['h3_address', 'hex_id', 'h3_index']:
            if c in df_e3.columns:
                df_e3.rename(columns={c: 'h3_id'}, inplace=True)
                break
        if 'h3_id' not in df_e3.columns and df_e3.index.name in ['h3_id', 'h3_index']:
            df_e3 = df_e3.reset_index()
            if 'h3_id' not in df_e3.columns: df_e3.rename(columns={'index': 'h3_id'}, inplace=True)

    # Seleciona apenas as colunas necessárias para o merge
    # (ID e a coluna de interesse)
    if 'e3_mar' not in df_e3.columns:
        print("ERRO: Coluna 'e3_mar' não encontrada no arquivo de input.")
        return
        
    df_e3_clean = df_e3[['h3_id', 'e3_mar']].copy()
    
    # Garante que os IDs sejam string para o merge funcionar
    df_base['h3_id'] = df_base['h3_id'].astype(str)
    df_e3_clean['h3_id'] = df_e3_clean['h3_id'].astype(str)

    print("3. Realizando Merge (Left Join)...")
    # Mantemos a base completa (left). Quem não tem dado de mar vira NaN (e depois 0).
    df_final = df_base.merge(df_e3_clean, on='h3_id', how='left')
    
    # Preenche NaNs com 0 (quem não cruzou é área sem risco ou interior)
    nulos = df_final['e3_mar'].isna().sum()
    df_final['e3_mar'] = df_final['e3_mar'].fillna(0).astype(int)
    
    print(f"   - Hexágonos do interior preenchidos com 0: {nulos}")
    print(f"   - Hexágonos com risco confirmado (1): {df_final['e3_mar'].sum()}")

    print(f"4. Salvando arquivo final: {output_final}")
    df_final.to_parquet(output_final)
    print("Concluído! Colunas no arquivo final:")
    print(df_final.columns.tolist())

if __name__ == "__main__":
    consolidar_h3_final()

1. Lendo base (e1, e2, e5, vul_int): ..\data\clean\h3\br\br_h3_res9_e125_vul_int.parquet
   - Total de hexágonos na base: 4893154
2. Lendo indicador E3 (Nível do Mar): ..\data\clean\h3\br\br_h3_res9_e3_mar.parquet
3. Realizando Merge (Left Join)...
   - Hexágonos do interior preenchidos com 0: 0
   - Hexágonos com risco confirmado (1): 4250
4. Salvando arquivo final: ..\data\clean\h3\br\br_h3_res9_e1235_vul_int.parquet
Concluído! Colunas no arquivo final:
['h3_id', 'qtd_enderecos', 'CD_MUN', 'NM_MUN', 'CD_SETOR', '__index_level_0__', 'e5_sec_norm', 'ind_vul', 'ind_int', 'e1_des_norm', 'e2_inu_norm', 'e3_mar']
